## JOB-A-THON - September 2021 - Store Sales Prediction problem

### Notebook setup

In [0]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from prophet import Prophet

In [0]:
# Setting pandas options for good visualization
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)
pd.set_option('display.precision', 4)
pd.set_option('display.float_format',  '{:,.2f}'.format)
pd.set_option('display.max_info_columns', 30)

# seaborn style for chart visualization
sns.set_style("whitegrid")

### Creating datasets

In [0]:
# loading test dataset
# File location and type
file_location = "/FileStore/tables/TRAIN.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
train = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

train = train.toPandas()

In [0]:
# loading test dataset
# File location and type
file_location = "/FileStore/tables/TEST_FINAL.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
test = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

test = test.toPandas()

### Data preprocessing

In [0]:
train.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,"7,011.84"
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,"51,789.12"
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,"36,868.20"
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,"19,715.16"
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,"45,614.52"


In [0]:
# renaming feature names
train = train.rename(columns={"#Order":"Num_Order"})

# Recoding Discount column
discount = {"Yes":1, "No":0}
train["Discount"] = train["Discount"].map(discount)

# changing variable type 
train["Date"] = pd.to_datetime(train["Date"])
# Rename columns
train = train.rename(columns = {"Date": "ds", "Sales": "y"})

# reordering columns of train dataset
train = train[["Store_id","ds","y","Holiday","Discount"]]

In [0]:
# Recoding Discount column
discount = {"Yes":1, "No":0}
test["Discount"] = test["Discount"].map(discount)

# changing variable type 
test["Date"] = pd.to_datetime(test["Date"])
# Rename columns
test = test.rename(columns = {"Date": "ds"})

# reordering columns of train dataset
test = test[["Store_id","ds","Holiday","Discount"]]

### Store wise forecast using Prophet

In [0]:
# creating function for store wise prediction
def store_forecast_fit(history_pd, new_pd, store_i):
  
  history_pd = history_pd[history_pd["Store_id"]==store_i]
  history_pd = history_pd.drop(["Store_id"], axis = 1)
  
  new_pd = new_pd[new_pd["Store_id"]==store_i]
  new_pd = new_pd.drop(["Store_id"], axis = 1)
  
  # instantiate the model, configure the parameters
  model = Prophet(
    interval_width=0.95,
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=True,
    seasonality_mode='multiplicative'
  )
  model.add_regressor('Holiday')
  model.add_regressor('Discount')
    
  # fit the model
  model.fit(history_pd)

  forecast = model.predict(new_pd)  # return predictions
  return forecast

In [0]:
store_id = train["Store_id"].unique().tolist()

output_df = pd.DataFrame()
for i in store_id:
  forecast = store_forecast_fit(train, test, i)
  forecast.loc[:,"Store_id"] = i
  output_df = pd.concat([output_df, forecast])

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java

In [0]:
output_df.head()

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Discount,Discount_lower,Discount_upper,Holiday,Holiday_lower,Holiday_upper,daily,daily_lower,daily_upper,extra_regressors_multiplicative,extra_regressors_multiplicative_lower,extra_regressors_multiplicative_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat,Store_id
0,2019-06-01,"33,784.61","17,490.94","48,551.70","33,784.61","33,784.61",0.00,0.00,0.00,0.00,0.00,0.00,-0.10,-0.10,-0.10,0.00,0.00,0.00,-0.02,-0.02,-0.02,0.11,0.11,0.11,-0.03,-0.03,-0.03,0.00,0.00,0.00,"33,253.93",1
1,2019-06-02,"33,790.13","17,988.03","50,645.90","33,790.13","33,790.13",0.00,0.00,0.00,0.00,0.00,0.00,-0.10,-0.10,-0.10,0.00,0.00,0.00,0.04,0.04,0.04,0.17,0.17,0.17,-0.04,-0.04,-0.04,0.00,0.00,0.00,"34,996.36",1
2,2019-06-03,"33,795.65","12,594.89","43,816.67","33,795.65","33,795.65",0.00,0.00,0.00,0.00,0.00,0.00,-0.10,-0.10,-0.10,0.00,0.00,0.00,-0.16,-0.16,-0.16,-0.01,-0.01,-0.01,-0.05,-0.05,-0.05,0.00,0.00,0.00,"28,527.74",1
3,2019-06-04,"33,801.16","8,581.95","41,320.75","33,801.16","33,801.16",0.00,0.00,0.00,0.00,0.00,0.00,-0.10,-0.10,-0.10,0.00,0.00,0.00,-0.26,-0.26,-0.26,-0.11,-0.11,-0.11,-0.05,-0.05,-0.05,0.00,0.00,0.00,"24,966.86",1
4,2019-06-05,"33,806.68","4,216.59","36,511.07","33,806.68","33,806.68",0.00,0.00,0.00,-0.23,-0.23,-0.23,-0.10,-0.10,-0.10,-0.23,-0.23,-0.23,-0.40,-0.40,-0.40,-0.01,-0.01,-0.01,-0.06,-0.06,-0.06,0.00,0.00,0.00,"20,237.64",1


In [0]:
test.head()

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java

,Store_id,ds,Holiday,Discount
0,171,2019-06-01,0,0
1,172,2019-06-01,0,0
2,173,2019-06-01,0,0
3,174,2019-06-01,0,0
4,170,2019-06-01,0,0


In [0]:
display(output_df)

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java

ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Discount,Discount_lower,Discount_upper,Holiday,Holiday_lower,Holiday_upper,daily,daily_lower,daily_upper,extra_regressors_multiplicative,extra_regressors_multiplicative_lower,extra_regressors_multiplicative_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat,Store_id
2019-06-01T00:00:00.000+0000,33784.613131163445,17490.935052123306,48551.70100183686,33784.61313048941,33784.613131163445,0.0,0.0,0.0,0.0,0.0,0.0,-0.10117724163420574,-0.10117724163420574,-0.10117724163420574,0.0,0.0,0.0,-0.015707829030116215,-0.015707829030116215,-0.015707829030116215,0.11243672042079528,0.11243672042079528,0.11243672042079528,-0.02696730781670574,-0.02696730781670574,-0.02696730781670574,0.0,0.0,0.0,33253.93020425051,1
2019-06-02T00:00:00.000+0000,33790.129516512265,17988.032811712546,50645.90264501987,33790.12951028085,33790.129525650496,0.0,0.0,0.0,0.0,0.0,0.0,-0.10117724163420574,-0.10117724163420574,-0.10117724163420574,0.0,0.0,0.0,0.03569784827752253,0.03569784827752253,0.03569784827752253,0.17405740607327141,0.17405740607327141,0.17405740607327141,-0.03718231616154314,-0.03718231616154314,-0.03718231616154314,0.0,0.0,0.0,34996.36443327056,1
2019-06-03T00:00:00.000+0000,33795.645901861084,12594.888189887584,43816.67311613849,33795.64588632462,33795.64592232192,0.0,0.0,0.0,0.0,0.0,0.0,-0.10117724163420574,-0.10117724163420574,-0.10117724163420574,0.0,0.0,0.0,-0.1558753205507946,-0.1558753205507946,-0.1558753205507946,-0.00860514524265969,-0.00860514524265969,-0.00860514524265969,-0.04609293367392918,-0.04609293367392918,-0.04609293367392918,0.0,0.0,0.0,28527.738763687343,1
2019-06-04T00:00:00.000+0000,33801.1622872099,8581.94966371522,41320.74965687631,33801.16225921899,33801.16232033162,0.0,0.0,0.0,0.0,0.0,0.0,-0.10117724163420574,-0.10117724163420574,-0.10117724163420574,0.0,0.0,0.0,-0.2613609955849347,-0.2613609955849347,-0.2613609955849347,-0.1066484519245054,-0.1066484519245054,-0.1066484519245054,-0.053535302026223516,-0.053535302026223516,-0.053535302026223516,0.0,0.0,0.0,24966.856859896772,1
2019-06-05T00:00:00.000+0000,33806.67867255872,4216.585394135987,36511.06682429673,33806.67863055101,33806.67872503374,0.0,0.0,0.0,-0.22795434202089854,-0.22795434202089854,-0.22795434202089854,-0.10117724163420574,-0.10117724163420574,-0.10117724163420574,-0.22795434202089854,-0.22795434202089854,-0.22795434202089854,-0.40137164803912717,-0.40137164803912717,-0.40137164803912717,-0.012877705152119697,-0.012877705152119697,-0.012877705152119697,-0.059362359231903195,-0.059362359231903195,-0.059362359231903195,0.0,0.0,0.0,20237.636339024615,1
2019-06-06T00:00:00.000+0000,33812.19505790754,8888.413833614395,41396.834072893165,33812.19499660251,33812.19512465653,0.0,0.0,0.0,0.0,0.0,0.0,-0.10117724163420574,-0.10117724163420574,-0.10117724163420574,0.0,0.0,0.0,-0.2493561474298768,-0.2493561474298768,-0.2493561474298768,-0.08473179448046528,-0.08473179448046528,-0.08473179448046528,-0.06344711131520583,-0.06344711131520583,-0.06344711131520583,0.0,0.0,0.0,25380.9163621202,1
2019-06-07T00:00:00.000+0000,33817.71144325637,9215.421220337674,40780.02871727234,33817.71136500033,33817.71152500351,0.0,0.0,0.0,0.0,0.0,0.0,-0.10117724163420574,-0.10117724163420574,-0.10117724163420574,0.0,0.0,0.0,-0.24049392559870722,-0.24049392559870722,-0.24049392559870722,-0.07363102969458837,-0.07363102969458837,-0.07363102969458837,-0.06568565426991313,-0.06568565426991313,-0.06568565426991313,0.0,0.0,0.0,25684.75726350332,1
2019-06-08T00:00:00.000+0000,33823.22782860519,31366.173720552953,62663.65476766586,33823.22772390022,33823.22792764053,0.4315230119935209,0.4315230119935209,0.4315230119935209,0.0,0.0,0.0,-0.10117724163420574,-0.10117724163420574,-0.10117724163420574,0.4315230119935209,0.4315230119935209,0.4315230119935209,0.3767826218740609,0.376782621874060

In [0]:
# creating submission file
submission_df = output_df[["Store_id","ds","yhat"]]
submission_df.columns = ["Store_id", "ds","Sales"]
submission_df.head()

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java

,Store_id,ds,Sales
0,1,2019-06-01,"33,253.93"
1,1,2019-06-02,"34,996.36"
2,1,2019-06-03,"28,527.74"
3,1,2019-06-04,"24,966.86"
4,1,2019-06-05,"20,237.64"


In [0]:
display(submission_df)

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java

Store_id,ds,Sales
1,2019-06-01T00:00:00.000+0000,33253.93020425051
1,2019-06-02T00:00:00.000+0000,34996.36443327056
1,2019-06-03T00:00:00.000+0000,28527.738763687343
1,2019-06-04T00:00:00.000+0000,24966.856859896772
1,2019-06-05T00:00:00.000+0000,20237.636339024615
1,2019-06-06T00:00:00.000+0000,25380.9163621202
1,2019-06-07T00:00:00.000+0000,25684.75726350332
1,2019-06-08T00:00:00.000+0000,46567.23229011075
1,2019-06-09T00:00:00.000+0000,48715.53844756546
1,2019-06-10T00:00:00.000+0000,42666.88235273548
